In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
CSV_COLUMN_NAMES=['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES=['Setosa', 'Versicolor', 'Virginica']

In [4]:
train=pd.read_csv("E:\Machine learning\Datasets\iris_training.csv", names=CSV_COLUMN_NAMES, header=0)
test=pd.read_csv("E:\Machine learning\Datasets\iris_test.csv", names=CSV_COLUMN_NAMES, header=0)

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
train_y=train.pop('Species')
test_y=test.pop('Species')

In [10]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [16]:
#def input_evaluation_set():
#features={'SepalLenght':np.array([6.4, 5.0]),
#'SepalWidth':np.array([2.8, 2.3]),
#'PetalLength':np.array([5.6, 3.3]),
#'PetalWidth':np.array([2.2, 1.0])}
# labels=np.array([2, 1])
#return features, labels

In [19]:
def input_fn(features, labels, training=True, batch_size=256):
    dataset=tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if training:
        dataset=dataset.shuffle(1000).repeat()
    return dataset.batch(batch_size)

In [20]:
my_feature_columns=[]
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [21]:
classifier=tf.estimator.DNNClassifier(feature_columns=my_feature_columns, hidden_units=[30, 10], n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\AYUSHS~1\\AppData\\Local\\Temp\\tmpw26kkmxd', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000287D9B6A308>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [22]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True), steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\AYUSHS~1\AppData\Local\Temp\tmpw26kkmxd\model.ckpt.
INFO:tensorflow:loss = 4.3544903, step = 0
INFO:tensorflow:global_step/sec: 140.219
INFO:tensorflow:loss = 0.177793, step = 100 (0.713 sec)
INFO:tensorflow:global_step/sec: 176.989
INFO:tensorflow:loss = 0.10453012, ste

In [23]:
eval_result=classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-07-14T12:42:50Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\AYUSHS~1\AppData\Local\Temp\tmpw26kkmxd\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-07-14-12:42:52
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.07608546, global_step = 5000, loss = 0.07608546
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\AYUSHS~1\AppData\Local\Temp\tmpw26kkmxd\model.ckpt-5000

Test set accuracy: 0.967



In [28]:
expected=['Setosa', 'Versicolor', 'Virginica']
predict_x={'SepalLength': [6.4, 5.0, 4.9],
    'SepalWidth': [2.5, 2.3, 2.8],
    'PetalLength': [4.5, 3.3, 5.6],
    'PetalWidth': [1.7, 1.0, 2.2],
}
def input_fn(features, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(
    input_fn=lambda: input_fn(predict_x))

In [29]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%), expected "{}"'.format(
        SPECIES[class_id], 100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\AYUSHS~1\AppData\Local\Temp\tmpw26kkmxd\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Versicolor" (74.5%), expected "Setosa"
Prediction is "Versicolor" (99.9%), expected "Versicolor"
Prediction is "Virginica" (100.0%), expected "Virginica"
